# 02 - Análisis de Modelo Individual

Este notebook demuestra cómo analizar un modelo individual usando la clase `AnalysisEngine`.

## Objetivos:
- Cargar un modelo desde archivo
- Ejecutar análisis estático
- Ejecutar análisis modal
- Visualizar resultados básicos

In [ ]:
import sys
sys.path.append('../src')

from analysis_engine import AnalysisEngine
from model_builder import ModelBuilder
from utils.model_helpers import ModelBuilderHelpers
import json
import os
import numpy as np

## 1. Inicializar el Motor de Análisis

In [ ]:
# Inicializar el motor de análisis
engine = AnalysisEngine(
    models_dir="../models",
    results_dir="../results"
)

print("Motor de análisis inicializado")
print(f"Directorio de modelos: {engine.models_dir}")
print(f"Directorio de resultados: {engine.results_dir}")

## 0. Crear Modelo de Prueba (Arquitectura Actualizada)

Primero creamos un modelo usando la nueva arquitectura modular:

In [ ]:
# Crear un modelo de prueba usando la arquitectura modular
builder = ModelBuilder(output_dir="../models")
helpers = ModelBuilderHelpers(builder)

# Crear modelo con configuración específica para demostración
test_model = builder.create_model(
    L_B_ratio=1.5, B=10.0, nx=3, ny=3,
    model_name="test_analisis_individual",
    enabled_analyses=['static', 'modal'],
    analysis_params={
        'static': {'steps': 15},
        'modal': {'num_modes': 8},
        'visualization': {
            'enabled': True,
            'static_deformed': True,
            'modal_shapes': True,
            'deform_scale': 150
        }
    }
)

print(f"✅ Modelo creado: {test_model['name']}")
print(f"📁 Archivo: {test_model['file_path']}")
print(f"🔧 Análisis configurados: {test_model.get('enabled_analyses', ['static', 'modal'])}")

# Este modelo será analizado en las siguientes secciones
MODEL_NAME = test_model['name']

## 2. Seleccionar un Modelo para Analizar

In [ ]:
# Obtener lista de modelos disponibles
model_files = engine.get_model_files()

print(f"Modelos disponibles: {len(model_files)}")
print("\nPrimeros 5 modelos:")
for i, file in enumerate(model_files[:5]):
    print(f"{i+1}. {os.path.basename(file)}")

# Seleccionar el primer modelo para análisis
if model_files:
    selected_model = model_files[0]
    print(f"\nModelo seleccionado: {os.path.basename(selected_model)}")
else:
    print("No hay modelos disponibles. Ejecuta primero el notebook 01_crear_modelos.ipynb")
    selected_model = None

## 3. Cargar y Verificar el Modelo

In [ ]:
if selected_model:
    # Cargar modelo
    model_data = engine.load_model_from_file(selected_model)
    
    print("=== INFORMACIÓN DEL MODELO ===")
    print(f"Nombre: {model_data['name']}")
    print(f"Parámetros:")
    for key, value in model_data['parameters'].items():
        print(f"  {key}: {value}")
    
    print(f"\nNodos: {len(model_data['nodes'])}")
    print(f"Elementos: {len(model_data['elements'])}")
    print(f"Cargas: {len(model_data['loads'])}")

## 4. Ejecutar Análisis Estático

In [ ]:
if selected_model:
    print("Ejecutando análisis estático...")
    
    # Ejecutar análisis estático
    static_results = engine.run_static_analysis(model_data)
    
    print("=== RESULTADOS DEL ANÁLISIS ESTÁTICO ===")
    print(f"Éxito: {static_results['success']}")
    
    if static_results['success']:
        print(f"Desplazamiento máximo: {static_results['max_displacement']:.6f} m")
        print(f"Base de datos ODB creada: {static_results['odb_data'] is not None}")
    else:
        print(f"Error: {static_results.get('error', 'Error desconocido')}")

## 5. Ejecutar Análisis Modal

In [ ]:
if selected_model:
    print("\nEjecutando análisis modal...")
    
    # Ejecutar análisis modal
    modal_results = engine.run_modal_analysis(model_data)
    
    print("=== RESULTADOS DEL ANÁLISIS MODAL ===")
    print(f"Éxito: {modal_results['success']}")
    
    if modal_results['success']:
        print(f"Número de modos calculados: {len(modal_results['frequencies'])}")
        print("\nFrecuencias y períodos:")
        for i, (freq, period) in enumerate(zip(modal_results['frequencies'], modal_results['periods'])):
            print(f"  Modo {i+1}: f = {freq:.4f} Hz, T = {period:.4f} s")
    else:
        print(f"Error: {modal_results.get('error', 'Error desconocido')}")

## 6. Análisis Completo del Modelo

In [ ]:
if selected_model:
    print("\nEjecutando análisis completo...")
    
    # Ejecutar análisis completo
    complete_results = engine.analyze_model(selected_model)
    
    print("=== RESULTADOS DEL ANÁLISIS COMPLETO ===")
    print(f"Modelo: {complete_results['model_name']}")
    print(f"Análisis estático exitoso: {complete_results['static_analysis']['success']}")
    print(f"Análisis modal exitoso: {complete_results['modal_analysis']['success']}")
    
    if complete_results['static_analysis']['success']:
        print(f"Desplazamiento máximo: {complete_results['static_analysis']['max_displacement']:.6f} m")
    
    if complete_results['modal_analysis']['success']:
        periods = complete_results['modal_analysis']['periods']
        print(f"Período fundamental: {periods[0]:.4f} s")
        print(f"Segundo período: {periods[1]:.4f} s" if len(periods) > 1 else "Solo un modo disponible")

## 7. Crear Visualización Básica

In [ ]:
if selected_model and complete_results['static_analysis']['success']:
    print("\nCreando visualización del modelo...")
    
    # Crear visualización
    html_file = engine.create_visualization(model_data)
    
    if html_file:
        print(f"Visualización guardada en: {html_file}")
        print("Abre el archivo HTML en tu navegador para ver el modelo")
    else:
        print("Error al crear la visualización")

## 8. Verificar Archivos de Resultados

In [ ]:
# Verificar archivos generados
if os.path.exists(engine.results_dir):
    result_files = os.listdir(engine.results_dir)
    
    print("=== ARCHIVOS DE RESULTADOS GENERADOS ===")
    print(f"Total de archivos: {len(result_files)}")
    
    for file in result_files:
        print(f"- {file}")
else:
    print("Directorio de resultados no existe")

## 9. Resumen de Verificación

In [ ]:
print("=== RESUMEN DE VERIFICACIÓN ===")
if selected_model:
    print(f"✓ Modelo analizado: {os.path.basename(selected_model)}")
    print(f"✓ Análisis estático: {'✓' if complete_results['static_analysis']['success'] else '✗'}")
    print(f"✓ Análisis modal: {'✓' if complete_results['modal_analysis']['success'] else '✗'}")
    print(f"✓ Archivos de resultados generados: {len(os.listdir(engine.results_dir)) if os.path.exists(engine.results_dir) else 0}")
    
    if complete_results['static_analysis']['success']:
        print(f"✓ Desplazamiento máximo calculado: {complete_results['static_analysis']['max_displacement']:.6f} m")
    
    if complete_results['modal_analysis']['success']:
        print(f"✓ Modos calculados: {len(complete_results['modal_analysis']['frequencies'])}")
    
    print("\n✅ Verificación del motor de análisis completada!")